In [1]:
import pandas as pd

In [85]:
import pandas as pd
from datasets import Dataset
from sklearn.utils import shuffle
import csv

def read_data(circa_data_path):
    df = pd.read_csv(circa_data_path, sep='\t')
    df = df.dropna()
    # df = df[:200]

    # prompt = pd.read_csv(prompt_templates_path, sep=';')
    return df

def prepare_dataset(df):
    new_context = df['context'].tolist() * 2
    new_question = df['question-X'].tolist() * 2
    new_answer = df['canquestion-X'].tolist() + df['answer-Y'].tolist()
    new_df = pd.DataFrame({'context': new_context, 'question': new_question, 'answer': new_answer})
    new_df['type'] = ['Direct answer'] * len(df) + ['Indirect answer'] * len(df)

    # label_counts = new_df['type'].value_counts()

    # print(label_counts,len(new_df))
    options = ['Direct answer','Indirect answer']
    # # from sklearn.utils import shuffle
    # # new_df = shuffle(new_df)
    # # random_examples = new_df.sample(4)
    # # print(random_examples)

    # # prompt_template_k_shot = prompt.loc[prompt['type'] == 'k-shot', 'prompt_template'].values[0]
    # # prompt_template_k_shot = prompt_template_k_shot.replace('\\n', '\n')
    new_df['options'] = [options] * len(new_df)
    new_df.rename(columns={'type': 'correct answer'}, inplace=True)
    new_df['pretext'] = new_df.apply(lambda x: f"Context: {x['context']}\nQuestion: {x['question']}\nResponse: {x['answer']}\n", axis=1)
    new_df.drop(['context','question','answer'],axis=1,inplace=True)
    return new_df


circa_data_path = './pragmatics/data/circa-data.tsv'
# prompt_templates_path = './prompt_templates/task_1.csv'
df = read_data(circa_data_path)
df = prepare_dataset(df)
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)


In [86]:
df['pretext'] = df['pretext'].str.strip()
len(df)

34790

In [87]:
df1 = pd.read_json("./pragmatics/human_data/task_0_human_dataset.json", orient='records')

In [88]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nContext")
    question = question[index+1:]
    question = question.replace("\nAnswer:","\nResponse:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops

In [93]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)

In [94]:
len(merged_df)

100

In [107]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_0_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_0_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)


In [110]:
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

In [130]:
import pandas as pd
from datasets import Dataset
from sklearn.utils import shuffle

def read_data(circa_data_path):
    df = pd.read_csv(circa_data_path, sep='\t')
    df = df.dropna()
    return df
# ,Context,Question,Answer,Implicature,goldstandard2
def prepare_dataset(df):

    # new_df = df.groupby['goldstandard2'].head(2000)
    new_df = df
    # print(label_counts)
    options = ['Yes','No','Yes, subject to some conditions','In the middle, neither yes nor no','Other']

    new_df['options'] = [options] * len(new_df)
    new_df.rename(columns={'goldstandard2': 'correct answer'}, inplace=True)
    new_df['pretext'] = new_df.apply(lambda x: f"Context: {x['context']}\nX: {x['question-X']}\nY: {x['answer-Y']}", axis=1)
    #new_df.drop(['Context','Question','Answer','Implicature'],axis=1,inplace=True)
    return new_df



circa_data_path = './pragmatics/data/circa-data.tsv'

df = read_data(circa_data_path)
df = prepare_dataset(df)
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
df

,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,correct answer,options,pretext
0,0,Y has just travelled from a different city to ...,Are you employed?,I am employed .,I'm a veterinary technician.,Yes#Yes#Yes#Yes#Yes,Yes,Yes,"[Yes, No, Yes, subject to some conditions, In ...",Context: Y has just travelled from a different...
1,1,X wants to know about Y's food preferences.,Are you a fan of Korean food?,I am a fan of Korean food .,I wouldn't say so,Probably no#No#No#No#Probably yes / sometimes yes,No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's food prefer...
2,2,Y has just told X that he/she is thinking of b...,Are you bringing any pets into the flat?,I am bringing pets into the flat .,I do not own any pets,No#No#No#No#No,No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: Y has just told X that he/she is thin...
3,3,X wants to know what activities Y likes to do ...,Would you like to get some fresh air in your f...,I would like to get fresh air in my free time .,I am desperate to get out of the city.,"Yes#Yes, subject to some conditions#Probably y...",Yes,Yes,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know what activities Y lik...
4,4,X and Y are childhood neighbours who unexpecte...,Is your family still living in the neighborhood?,My family is living in the neighborhood .,My parents are snowbirds now.,"No#In the middle, neither yes nor no#Probably ...","In the middle, neither yes nor no","In the middle, neither yes nor no","[Yes, No, Yes, subject to some conditions, In ...",Context: X and Y are childhood neighbours who ...
...,...,...,...,...,...,...,...,...,...,...
31355,34263,X wants to know what activities Y likes to do ...,Do you like to drink?,I like to drink .,I am in AA.,No#No#No#Probably no#No,No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know what activities Y lik...
31356,34264,X wants to know about Y's food preferences.,Do you like pie?,I like pie .,My favorite pie is pecan.,"Yes#Yes#Yes, subject to some conditions#Yes#Yes",Yes,Yes,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's food prefer...
31357,34265,X wants to know about Y's music preferences.,Want to go to a concert with me?,I want to go to a concert with me .,I'd rather do something else.,"No#In the middle, neither yes nor no#Probably ...",No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's music prefe...
31358,34266,X wants to know about Y's music preferences.,Do you like hip/hop music?,I like hip/hop music .,I can't dance to hip/hop music,"Probably no#Probably no#In the middle, neither...",Probably no,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's music prefe...


In [131]:
df1 = pd.read_json("./pragmatics/human_data/task_1_human_dataset.json", orient='records')

In [132]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nContext")
    question = question[index+1:]
    question = question.replace("\nQuestion:","\nX:")
    question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops

In [133]:
df1

,text,options
0,Context: X and Y are colleagues who are leavin...,"[Yes, No, Yes, subject to some conditions, In ..."
1,Context: X wants to know what activities Y lik...,"[Yes, No, Yes, subject to some conditions, In ..."
2,Context: X and Y are childhood neighbours who ...,"[Yes, No, Yes, subject to some conditions, In ..."
3,Context: Y has just moved into a neighbourhood...,"[Yes, No, Yes, subject to some conditions, In ..."
4,Context: X wants to know what activities Y lik...,"[Yes, No, Yes, subject to some conditions, In ..."
...,...,...
95,Context: Y has just told X that he/she is thin...,"[Yes, No, Yes, subject to some conditions, In ..."
96,Context: Y has just moved into a neighbourhood...,"[Yes, No, Yes, subject to some conditions, In ..."
97,Context: X wants to know about Y's music prefe...,"[Yes, No, Yes, subject to some conditions, In ..."
98,Context: Y has just travelled from a different...,"[Yes, No, Yes, subject to some conditions, In ..."


In [134]:
df

,id,context,question-X,canquestion-X,answer-Y,judgements,goldstandard1,correct answer,options,pretext
0,0,Y has just travelled from a different city to ...,Are you employed?,I am employed .,I'm a veterinary technician.,Yes#Yes#Yes#Yes#Yes,Yes,Yes,"[Yes, No, Yes, subject to some conditions, In ...",Context: Y has just travelled from a different...
1,1,X wants to know about Y's food preferences.,Are you a fan of Korean food?,I am a fan of Korean food .,I wouldn't say so,Probably no#No#No#No#Probably yes / sometimes yes,No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's food prefer...
2,2,Y has just told X that he/she is thinking of b...,Are you bringing any pets into the flat?,I am bringing pets into the flat .,I do not own any pets,No#No#No#No#No,No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: Y has just told X that he/she is thin...
3,3,X wants to know what activities Y likes to do ...,Would you like to get some fresh air in your f...,I would like to get fresh air in my free time .,I am desperate to get out of the city.,"Yes#Yes, subject to some conditions#Probably y...",Yes,Yes,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know what activities Y lik...
4,4,X and Y are childhood neighbours who unexpecte...,Is your family still living in the neighborhood?,My family is living in the neighborhood .,My parents are snowbirds now.,"No#In the middle, neither yes nor no#Probably ...","In the middle, neither yes nor no","In the middle, neither yes nor no","[Yes, No, Yes, subject to some conditions, In ...",Context: X and Y are childhood neighbours who ...
...,...,...,...,...,...,...,...,...,...,...
31355,34263,X wants to know what activities Y likes to do ...,Do you like to drink?,I like to drink .,I am in AA.,No#No#No#Probably no#No,No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know what activities Y lik...
31356,34264,X wants to know about Y's food preferences.,Do you like pie?,I like pie .,My favorite pie is pecan.,"Yes#Yes#Yes, subject to some conditions#Yes#Yes",Yes,Yes,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's food prefer...
31357,34265,X wants to know about Y's music preferences.,Want to go to a concert with me?,I want to go to a concert with me .,I'd rather do something else.,"No#In the middle, neither yes nor no#Probably ...",No,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's music prefe...
31358,34266,X wants to know about Y's music preferences.,Do you like hip/hop music?,I like hip/hop music .,I can't dance to hip/hop music,"Probably no#Probably no#In the middle, neither...",Probably no,No,"[Yes, No, Yes, subject to some conditions, In ...",Context: X wants to know about Y's music prefe...


In [135]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)

In [136]:
len(merged_df)

100

In [138]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_1_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_1_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 2

In [19]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_2_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_2.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)

In [20]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nContext")
    question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Context: X wants to know what sorts of books Y...,"[Yes, No, Yes, subject to some conditions, In ..."
1,Context: X wants to know what activities Y lik...,"[Yes, No, Yes, subject to some conditions, In ..."
2,Context: Y has just travelled from a different...,"[Yes, No, Yes, subject to some conditions, In ..."
3,Context: X wants to know what sorts of books Y...,"[Yes, No, Yes, subject to some conditions, In ..."
4,Context: X wants to know about Y's food prefer...,"[Yes, No, Yes, subject to some conditions, In ..."
...,...,...
95,Context: Y has just travelled from a different...,"[Yes, No, Yes, subject to some conditions, In ..."
96,Context: X wants to know about Y's music prefe...,"[Yes, No, Yes, subject to some conditions, In ..."
97,Context: X and Y are colleagues who are leavin...,"[Yes, No, Yes, subject to some conditions, In ..."
98,Context: X wants to know about Y's music prefe...,"[Yes, No, Yes, subject to some conditions, In ..."


In [21]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [23]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_2_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_2_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 3

In [6]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_3_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_3.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

101 2000


In [7]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\n\nSpeaker_1:")
    question = question[index+2:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Speaker_1: how can I find Noah\nSpeaker_2: he ...,"[all the turnips are in the lounge, five of th..."
1,Speaker_1: where can I find the spinaches\nSpe...,"[Aria was in the hallway, I was in the master_..."
2,Speaker_1: did you leave the tomato in the bas...,"[I was not in the basement, I am not sure if E..."
3,Speaker_1: where can I find the spinaches\nSpe...,"[five of the tomatoes are in the lounge, all o..."
4,Speaker_1: did you see the sweet potato\nSpeak...,"[four of the cabbages are in the cellar, all o..."
...,...,...
96,Speaker_1: did you see the plums\nSpeaker_2: I...,"[Liam didn't place the corns in the office, th..."
97,Speaker_1: are some of the cabbages in the bed...,"[I don't know, Benjamin was in the patio, Benj..."
98,Speaker_1: did Ethan leave the corn in the lou...,"[I was not in the staircase, I don't know, Eth..."
99,Speaker_1: are some of the onions in the laund...,"[I put the strawberries in the laundry, I put ..."


In [8]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

101

In [9]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_3_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_3_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 4

In [14]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_4_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_4.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

100 1999


In [15]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nSpeaker_1:")
    question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Speaker_1: Life is simply a board game of Risk...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
1,Speaker_1: The braces on his teeth were ice\nS...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
2,Speaker_1: She's about as hot as a supermodel\...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
3,Speaker_1: Jacob's novel has the depth of a ki...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
4,Speaker_1: The girl's attitude is as sweet as ...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
...,...,...
95,Speaker_1: Reading the book is a fun little jo...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
96,Speaker_1: My ant bite has the itch of a crab ...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
97,Speaker_1: He divorced his wife because she be...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."
98,Speaker_1: Her status was as up in the air as ...,"[Speaker_2 agrees with Speaker_1, Speaker_2 di..."


In [16]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [17]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_4_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_4_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 5

In [18]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_5_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_5.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

100 2000


In [19]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nSpeaker_1:")
    question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Speaker_1: Life is full of wonder and color wh...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
1,Speaker_1: Braces offer a strength and securit...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
2,Speaker_1: She is not good looking.\nSpeaker_2...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
3,Speaker_1: the book is inspiring.\nSpeaker_2: ...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
4,Speaker_1: The girl's attitude is a bit tart.\...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
...,...,...
95,"Speaker_1: The book is a quick, entertaining r...","[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
96,Speaker_1: My ant bite isn't that itchy\nSpeak...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
97,Speaker_1: He divorced his wife because she be...,"[Speaker_2 agrees with Speaker_1, Speaker_2 is..."
98,"Speaker_1: it was not sure\nSpeaker_2: Yes, He...","[Speaker_2 agrees with Speaker_1, Speaker_2 is..."


In [20]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [21]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_5_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_5_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 6

In [22]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_6_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_6.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

100 1767


In [23]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nSentence :")
    question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Sentence : The man still had heavy feelings ab...,[The man still had unpleasant and negative fee...
1,Sentence : Are those people-the kind of people...,[Are those people-the kind of people who would...
2,"Sentence : So I hauled him in, and quent put h...","[So I hauled him in, and quent put him in dete..."
3,Sentence : The hollow barking of a dog came fr...,[The muffled barking of a dog came from one of...
4,Sentence : From the moment he'd crawled out of...,[From the moment he'd crawled out of bed earli...
...,...,...
95,Sentence : His devotion to his sick wife conde...,[He was so devoted to his sick wife that he ea...
96,Sentence : He felt like a great oak tree with ...,"[He felt weak and flimsy, He felt solid and st..."
97,Sentence : Either they go native and forget th...,[Either they adopt the behavior attitudes or c...
98,Sentence : this chair is as fragile as a brick,"[This chair is quite sturdy, This chair is fra..."


In [24]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [25]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_6_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_6_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 7

In [26]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_7_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_7.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

100 1769


In [27]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nSentence :")
    question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Sentence : The man still had heavy feelings ab...,[The man still had unpleasant and negative fee...
1,Sentence : Are those people-the kind of people...,[Are those people-the kind of people who would...
2,"Sentence : So I hauled him in, and quent put h...","[So I hauled him in, and quent put him in dete..."
3,Sentence : The hollow barking of a dog came fr...,[The muffled barking of a dog came from one of...
4,Sentence : From the moment he'd crawled out of...,[From the moment he'd crawled out of bed earli...
...,...,...
95,Sentence : His devotion to his sick wife conde...,[He was so devoted to his sick wife that he ea...
96,Sentence : He felt like a great oak tree with ...,"[He felt weak and flimsy, He felt solid and st..."
97,Sentence : Either they go native and forget th...,[Either they adopt the behavior attitudes or c...
98,Sentence : this chair is as fragile as a brick...,"[This chair is quite sturdy, This chair is fra..."


In [28]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [29]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_7_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_7_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 8

In [30]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_8_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_8.csv")
df['pretext'] = df['pretext'].str.strip()
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

100 1769


In [31]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\nSentence :")
    question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip()
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Sentence : The man still had heavy feelings ab...,[The man still had unpleasant and negative fee...
1,Sentence : Are those people-the kind of people...,[Are those people-the kind of people who would...
2,"Sentence : So I hauled him in, and quent put h...","[So I hauled him in, and quent put him in dete..."
3,Sentence : The hollow barking of a dog came fr...,[The muffled barking of a dog came from one of...
4,Sentence : From the moment he'd crawled out of...,[From the moment he'd crawled out of bed earli...
...,...,...
95,Sentence : His devotion to his sick wife conde...,[He was so devoted to his sick wife that he ea...
96,Sentence : He felt like a great oak tree with ...,"[He felt weak and flimsy, He felt solid and st..."
97,Sentence : Either they go native and forget th...,[Either they adopt the behavior attitudes or c...
98,Sentence : this chair is as fragile as a brick...,"[This chair is quite sturdy, This chair is fra..."


In [32]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [33]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_8_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_8_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 9

In [49]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_9_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_9.csv")
df['pretext'] = df['pretext'].str.strip('\n')
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

99 2100


In [50]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    #index = question.find("\nSentence :")
    #question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip('\n')
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Premise: Two hospitals judged Sarah to worry W...,"[Hypothesis is definitely true given premise, ..."
1,Premise: That skirt is great.\nHypothesis: Tha...,"[Hypothesis is definitely true given premise, ..."
2,Premise: Ten guests didn't research the Balkan...,"[Hypothesis is definitely true given premise, ..."
3,Premise: Russell hasn't researched two documen...,"[Hypothesis is definitely true given premise, ..."
4,Premise: Some actresses had longed to annoy th...,"[Hypothesis is definitely true given premise, ..."
...,...,...
94,Premise: Ten socks wouldn't astound Raymond.\n...,"[Hypothesis is definitely true given premise, ..."
95,Premise: Those actresses didn't tour one hundr...,"[Hypothesis is definitely true given premise, ..."
96,Premise: One hundred libraries didn't criticiz...,"[Hypothesis is definitely true given premise, ..."
97,Premise: Todd didn't annoy ten children.\nHypo...,"[Hypothesis is definitely true given premise, ..."


In [51]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

99

In [52]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_9_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_9_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 10

In [53]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_10_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_10.csv")
df['pretext'] = df['pretext'].str.strip('\n')
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

99 1800


In [54]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    #index = question.find("\nSentence :")
    #question = question[index+1:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip('\n')
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Premise: Does Brian's popsicle that couldn't s...,"[Hypothesis is definitely true given premise, ..."
1,Premise: Had the seven women who shouldn't res...,"[Hypothesis is definitely true given premise, ..."
2,Premise: The nine adults who ascertained Carme...,"[Hypothesis is definitely true given premise, ..."
3,Premise: Those waitresses' shoe that hadn't an...,"[Hypothesis is definitely true given premise, ..."
4,Premise: This door hadn't opened.\nHypothesis:...,"[Hypothesis is definitely true given premise, ..."
...,...,...
94,Premise: Kirsten doesn't only research politic...,"[Hypothesis is definitely true given premise, ..."
95,Premise: Does Kristin wonder how Janice has th...,"[Hypothesis is definitely true given premise, ..."
96,Premise: If the student's box that would resem...,"[Hypothesis is definitely true given premise, ..."
97,Premise: Marla might be firing those guests.\n...,"[Hypothesis is definitely true given premise, ..."


In [55]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

99

In [56]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_10_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_10_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 11

In [1]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_11_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_11.csv")
df['pretext'] = df['pretext'].str.strip('\n')
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

900 1000


In [2]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\n\nConversation:")
    question = question[index+2:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip('\n')
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Conversation:\nSpeaker_1: did you go to the ba...,"[yes, no]"
1,Conversation:\nSpeaker_1: did you put the cher...,"[yes, no]"
2,Conversation:\nSpeaker_1: are all the bananas ...,"[yes, no]"
3,Conversation:\nSpeaker_1: how many beans are i...,"[yes, no]"
4,Conversation:\nSpeaker_1: how many grapes are ...,"[yes, no]"
...,...,...
895,Conversation:\nSpeaker_1: did you see the pers...,"[yes, no]"
896,Conversation:\nSpeaker_1: did Nathan go to the...,"[yes, no]"
897,Conversation:\nSpeaker_1: how many sweet potat...,"[yes, no]"
898,Conversation:\nSpeaker_1: did Emily leave the ...,"[yes, no]"


In [3]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

900

In [ ]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_11_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_11_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 13

In [58]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_13_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_13.csv")
df['pretext'] = df['pretext'].str.strip('\n')
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

100 2563


In [59]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\n\nConversation:")
    question = question[index+2:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip('\n')
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,"Conversation:\nA: Excuse me , but I'm looking ...","[Valid, Invalid]"
1,Conversation:\nA: Good morning . How are you ?...,"[Valid, Invalid]"
2,Conversation:\nA: Now I'm sure we'll never get...,"[Valid, Invalid]"
3,"Conversation:\nA: Hello , this is United Airli...","[Valid, Invalid]"
4,"Conversation:\nA: Mary , today a man from an i...","[Valid, Invalid]"
...,...,...
95,Conversation:\nA: I'm glad we came here . This...,"[Valid, Invalid]"
96,Conversation:\nA: Have you had a chance to loo...,"[Valid, Invalid]"
97,"Conversation:\nA: Hello , Doctor . \nB: Well ...","[Valid, Invalid]"
98,Conversation:\nA: Is that room service ?\nB: Y...,"[Valid, Invalid]"


In [60]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

100

In [61]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_13_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_13_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)

# Task 14

In [64]:
import pandas as pd
df1 = pd.read_json("./pragmatics/human_data/task_14_human_dataset.json", orient='records')
df = pd.read_csv("./pragmatics/global_datasets/task_14.csv")
df['pretext'] = df['pretext'].str.strip('\n')
df = df.drop_duplicates(subset=['pretext']).reset_index(drop=True)
print(len(df1),len(df))

101 456


In [65]:
qs = []
ops = []
for i in range(len(df1)):
    question = df1['data'][i]['text']
    index = question.find("\n\nContext:")
    question = question[index+2:]
    #question = question.replace("\nQuestion:","\nX:")
    #question = question.replace("\nAnswer:","\nY:")
    question = question.strip('\n')
    options = df1['data'][i]['options']
    opts = [j['value'] for j in options]
    qs.append(question)
    ops.append(opts)
df1 = pd.DataFrame()
df1['text'] = qs
df1['options'] = ops
df1

,text,options
0,Context: The ball hit me on the head \nQuestio...,"[The forehead , The back of the head , The nec..."
1,Context: The chisel sculpted the masterpiece\n...,"[blade, Knife, pluck, sculptor]"
2,Context: Mumbai's Dalal Street reacts to chang...,"[A street with market stalls, A type of street..."
3,Context: The orange shoes does not look like a...,"[Someone wearing orange shoes, Footwear, Feet..."
4,Context: I can see your point \nQuestion: What...,"[opinion , objection , potential , all of the ..."
...,...,...
96,Context: She is proud of her wardrobe \nQuesti...,"[Dresses , hygiene , design, cupboard ]"
97,Context: The industry is facing challenges.\nQ...,"[Machinery , Factory, Factory workers, Busines..."
98,Context: We need some muscles to lift the furn...,"[Body builders , Gymnastics , Strong people , ..."
99,Context: The sax is ill today \nQuestion: What...,"[The saxophone , The instrument , saxophonist ..."


In [66]:
merged_df = df1.merge(df, left_on='text', right_on='pretext', how='inner',sort=False)
len(merged_df)

101

In [67]:
import json

# Define the input and output file paths
input_file_path = './pragmatics/human_data/task_14_human_dataset.json'
output_file_path = './pragmatics/human_answers/task_14_human_dataset.json'

# Define the key and value you want to add to each JSON object

# Initialize a list to store the modified JSON objects
modified_json_objects = []
j = 0
# Read the input JSON file line by line and add the new key-value pair
with open(input_file_path, 'r') as json_file:
    a = json.load(json_file)
    
for i in range(len(a)):
    a[i]['answer'] = merged_df['correct answer'][i]
    
with open(output_file_path, 'w') as json_file1:
    json.dump(a, json_file1)